# Certificate Editor

This notebook uses [cryptography](https://cryptography.io/) to take an existing X.509 certificate and make edits to it, while preserving the other characteristics of the certificate. In particular, this preserves the currently existing certificate private key.

By default, this code takes a certificate and changes its expiry date to be thirty years in the future from today, but in principle it can make other changes.

The script has the following limitations:

- It only works for self-signed certificates. This isn't a fundamental limitation, just done for simplicity's sake.

In [1]:
# Imports and other basic setup.
import datetime

from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization


THIRTY_YEARS = datetime.timedelta(days=365*30)

In [7]:
# User-controlled settings. These settings adjust what certificate is used and what gets edited about it.
CERT_PATH = "client.pem"
KEY_PATH = "client.key"
KEY_PASSWORD = None
OUTPUT_PATH = "newcert.pem"

NEW_PARAMETERS = {
    "not_valid_after": datetime.datetime.today() + THIRTY_YEARS
}
NEW_EXTENSIONS = {}

In [5]:
# The actual logic of the code. First, define all the certificate attributes we'll try to set.
# We deliberately exclude extensions here because they require more work.
# The attributes are a dictionary of "certificate builder name" to "certificate name", to work
# around the fact that cryptography has different names for most of these things,
attrs = {
    'subject_name': 'subject',
    'issuer_name': 'issuer',
    'not_valid_before': 'not_valid_before',
    'not_valid_after': 'not_valid_after', 
    'serial_number': 'serial',
}


def build_certificate(current_cert, current_key):
    builder = x509.CertificateBuilder()
    
    # Apply the attributes. We have to do a weird getattr() dance here becuase the various
    # builder attributes are actually functions.
    for attr in attrs:
        if attr not in NEW_PARAMETERS:
            try:
                old_attr = getattr(current_cert, attrs[attr])
            except AttributeError:
                continue
            else:
                builder = getattr(builder, attr)(old_attr)
        else:
            builder = getattr(builder, attr)(NEW_PARAMETERS[attr])
    
    # Then the extensions. First copy across the ones that are there, editing if needed.
    for extension in current_cert.extensions:
        if extension.value.__class__ not in NEW_EXTENSIONS:
            builder = builder.add_extension(extension.value, critical=extension.critical)
        else:
            builder = builder.add_extension(NEW_EXTENSIONS[extension])
            del NEW_EXTENSIONS[extension]
            
    # Then set any extra new extensions.
    for extension, value in NEW_EXTENSIONS.items():
        builder = builder.add_extension(value)
        
    # Finally, set the key and sign it.
    builder = builder.public_key(current_key.public_key())
    
    signature_algorithm = current_cert.signature_hash_algorithm
    return builder.sign(
        private_key=current_key,
        algorithm=signature_algorithm,
        backend=default_backend(),
    )

In [6]:
# Finally, a code block that actually executes the transformation.
with open(CERT_PATH, 'rb') as f:
    current_cert = x509.load_pem_x509_certificate(f.read(), default_backend())
    
with open(KEY_PATH, 'rb') as f:
    current_key = serialization.load_pem_private_key(f.read(), KEY_PASSWORD, default_backend())
    
new_cert = build_certificate(current_cert, current_key)
with open(OUTPUT_PATH, 'wb') as f:
    f.write(new_cert.public_bytes(serialization.Encoding.PEM))